In [ ]:
print('jmd shree ganesha')

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain_community.vectorstores import chroma
from langchain_community.llms import llamacpp
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

import os

In [ ]:
dataset = r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot\FAQ-Chatbot\data"

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question. You are a helpful assistant who answers questions related to Nuffield Health Hospital.
If the question is not related to Nuffield Health, respond with "I don't know, It's out of my knowledge base.", 
else if you don't know the answer, just say that to contact the helpline number 0330 173 8200. Not to ask for the phone number.

Context: {context}

Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf(dataset)

In [ ]:
def text_splitter(data_pdf):
    splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=10)
    chunks = splitter.split_documents(data_pdf)
    
    return chunks

In [ ]:
text_split_chunks = text_splitter(extracted_data)

In [ ]:
text_split_chunks

In [ ]:
print(len(text_split_chunks))

In [ ]:
def text_to_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = text_to_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
query_result

In [ ]:
persist_directory = 'chat_db'

vectordb = chroma.Chroma.from_documents(documents=text_split_chunks, embedding=embeddings, persist_directory=persist_directory)

In [ ]:
vectordb.persist()
vectordb = None

In [ ]:
vectordb = chroma.Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docsearch = retriever.get_relevant_documents('Do I need a referral from my GP?')

In [ ]:
len(docsearch)
print(docsearch)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={'k':4})

In [ ]:
llm2 = llamacpp.LlamaCpp(model_path=r"C:\Users\Admin\Desktop\Aster\FAQ Chatbot-ChromaDB\model\llama-2-7b-chat.ggmlv3.q4_0.bin", n_ctx=4096, n_gpu_layers=-1, n_threads=4, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0.5, context_length=4096, max_tokens=2048)

In [ ]:
llm_chain = LLMChain(llm=llm2, prompt=prompt_template)

In [ ]:
llm_chain = LLMChain.invoke()